In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


In [2]:
df = pd.read_csv('pima-indians-diabetes-database/diabetes.csv')

In [3]:
%matplotlib inline

In [4]:
df1 = df.loc[df['Outcome']==1]
df2 = df.loc[df['Outcome']==0]

In [5]:
df1 = df1.replace({'BloodPressure':0}, np.median(df1['BloodPressure']))
df2 = df2.replace({'BloodPressure':0}, np.median(df2['BloodPressure']))
data = [df1,df2]
df = pd.concat(data)

In [6]:
df1 = df.loc[df['Outcome']==1]
df2 = df.loc[df['Outcome']==0]

In [7]:
df1 = df1.replace({'BMI':0}, np.mean(df1['BMI']))
df2 = df2.replace({'BMI':0}, np.mean(df2['BMI']))

In [8]:
data = [df1,df2]
df = pd.concat(data)

In [9]:
df1 = df.loc[df['Outcome']==1]
df2 = df.loc[df['Outcome']==0]

In [10]:
df1 = df1.replace({'Glucose':0}, np.mean(df1['Glucose']))
df2 = df2.replace({'Glucose':0}, np.mean(df2['Glucose']))

In [11]:
data = [df1,df2]
df = pd.concat(data)

In [12]:
mean_insulin_diabetic = df['Insulin'].loc[df['Outcome']==1].sum()/130
mean_insulin_diabetic.round()

207.0

In [13]:
mean_insulin_non_diabetic = df['Insulin'].loc[df['Outcome']==0].sum()/264
mean_insulin_non_diabetic.round()

130.0

In [14]:
df1 = df.loc[df['Outcome']==1]
df2 = df.loc[df['Outcome']==0]

In [15]:
df1 = df1.replace({'Insulin':0}, mean_insulin_diabetic)
df2 = df2.replace({'Insulin':0}, mean_insulin_non_diabetic)

In [16]:
data = [df1,df2]
df = pd.concat(data)

In [17]:
mean_diabetic_skinThickness = df['SkinThickness'].loc[df['Outcome']==1].sum()/180
mean_diabetic_skinThickness.round()

33.0

In [18]:
mean_nondiabetic_skinThickness = df['SkinThickness'].loc[df['Outcome']==0].sum()/361
mean_nondiabetic_skinThickness.round()

27.0

In [19]:
df1 = df.loc[df['Outcome']==1]
df2 = df.loc[df['Outcome']==0]

In [20]:
df1 = df1.replace({'SkinThickness':0}, mean_diabetic_skinThickness)
df2 = df2.replace({'SkinThickness':0}, mean_nondiabetic_skinThickness)

In [21]:
data = [df1,df2]
df = pd.concat(data)

In [42]:
df.Outcome.unique()

array([1, 0], dtype=int64)

### ML section classification without using smote

In [23]:
from sklearn.model_selection import train_test_split

In [24]:
X = df.drop('Outcome', axis=1).values
y = df['Outcome'].values

In [25]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=101)

In [26]:
from sklearn.preprocessing import StandardScaler

In [27]:
scaler = StandardScaler()

In [28]:
X_train = scaler.fit_transform(X_train)

In [29]:
X_test = scaler.transform(X_test)

In [30]:
from matplotlib.colors import ListedColormap
def plot_decision_regions(X, y, classifier, resolution=0.02):
    
    # setup marker generator and color map
    markers = ('s', 'x', 'o', '^', 'v')
    colors = ('red', 'blue', 'lightgreen', 'gray', 'cyan')
    cmap = ListedColormap(colors[:len(np.unique(y))])
    
    # plot the decision surface
    x1_min, x1_max = X[:, 0].min() - 1, X[:, 0].max() + 1
    x2_min, x2_max = X[:, 1].min() - 1, X[:, 1].max() + 1
    xx1, xx2 = np.meshgrid(np.arange(x1_min, x1_max, resolution),
                           np.arange(x2_min, x2_max, resolution))
    Z = classifier.predict(np.array([xx1.ravel(), xx2.ravel()]).T)
    Z = Z.reshape(xx1.shape)
    plt.contourf(xx1, xx2, Z, alpha=0.4, cmap=cmap)
    plt.xlim(xx1.min(), xx1.max())
    plt.ylim(xx2.min(), xx2.max())
    
    # plot examples by class
    for idx, cl in enumerate(np.unique(y)):
        plt.scatter(x=X[y == cl, 0],
                    y=X[y == cl, 1],
                    alpha=0.6,
                    color=cmap(idx),
                    edgecolor='black',
                    marker=markers[idx],
                    label=cl)




### Logistic Regression

In [31]:
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression()
logreg.fit(X_train, y_train)

LogisticRegression()

In [32]:
plot_decision_regions(X_train, y_train, classifier=logreg)
plt.legend(loc='upper left')
plt.tight_layout()
plt.show()

ValueError: X has 2 features per sample; expecting 8

In [36]:
predictions = logreg.predict(X_test)

In [37]:
print('Misclassified examples: %d' %(y_test!=predictions).sum())

Misclassified examples: 48


In [38]:
print('Accuracy of logistic regression classifier on test set: {:.2f}'.format(logreg.score(X_test, y_test)))

Accuracy of logistic regression classifier on test set: 0.79


In [34]:
from sklearn.metrics import classification_report,confusion_matrix

In [39]:
print(classification_report(y_test,predictions))

              precision    recall  f1-score   support

           0       0.80      0.90      0.84       145
           1       0.78      0.62      0.69        86

    accuracy                           0.79       231
   macro avg       0.79      0.76      0.77       231
weighted avg       0.79      0.79      0.79       231



In [ ]:
confusion_matrix(y_test,predictions)

### Model Fitting: Support Vector Machine (Kernel: rbf)

In [ ]:
from sklearn.svm import SVC
classifier_rbf = SVC(kernel = 'rbf')
classifier_rbf.fit(X_train, y_train)
predictions = classifier_rbf.predict(X_test)

In [ ]:
print('Accuracy of SVC (RBF) classifier on test set: {:.2f}'.format(classifier_rbf.score(X_test, y_test)))

In [ ]:
print(classification_report(y_test,predictions))

In [ ]:
confusion_matrix(y_test,predictions)

### Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(n_estimators=300, bootstrap = True, max_features = 'sqrt')
model.fit(X_train, y_train)

In [ ]:
predictions = model.predict(X_test)
print('Accuracy of Random Forest on test set: {:.2f}'.format(model.score(X_test, y_test)))

In [ ]:
print(classification_report(y_test,predictions))

In [ ]:
confusion_matrix(y_test,predictions)